# `OutputAdapter`


In [1]:
%load_ext autoreload
%autoreload 2

## On its own


In [2]:
from haystack import Document
from haystack.components.converters import OutputAdapter

In [3]:
adapter = OutputAdapter(template="{{ documents[0].content }}", output_type=str)
input_data = {"documents": [Document(content="Test content")]}
expected_output = {"output": "Test content"}
assert adapter.run(**input_data) == expected_output

## In Pipeline

In [4]:
from haystack import Pipeline, component, Document
from haystack.components.converters import OutputAdapter

@component
class DocumentProducer:
    @component.output_types(documents=dict)
    def run(self):
        return {"documents": [Document(content="haystack")]}

pipe = Pipeline()
pipe.add_component(
    name="output_adapter",
    instance=OutputAdapter(template="{{ documents[0].content | capitalize}}", output_type=str),
)
pipe.add_component(name="document_producer", instance=DocumentProducer())
pipe.connect("document_producer", "output_adapter")
result = pipe.run(data={})

assert result["output_adapter"]["output"] == "Haystack"

In [5]:
from haystack import Pipeline, component, Document
from haystack.components.converters import OutputAdapter

def reverse_string(s):
    return s[::-1]

@component
class DocumentProducer:
    @component.output_types(documents=dict)
    def run(self):
        return {"documents": [Document(content="haystack")]}

pipe = Pipeline()
pipe.add_component(
    name="output_adapter",
    instance=OutputAdapter(template="{{ documents[0].content | reverse_string}}",
                           output_type=str,
                           custom_filters={"reverse_string": reverse_string}))

pipe.add_component(name="document_producer", instance=DocumentProducer())
pipe.connect("document_producer", "output_adapter")
result = pipe.run(data={})

assert result["output_adapter"]["output"] == "kcatsyah"